## Alisa Todorova

# Introduction to lattice-based encryption

## Tasks
1. Write the LWE public-key encryption function
2. Write the RLWE public-key encryption and decryption functions.
3. Write the attack recovering the secret $s$ from the knowledge of $a$, $t$ and $e$.

## LWE encryption

We first recall basic lattice-based encryption, starting from the Regev scheme [Reg05].

Let $q \in {\mathbb Z}$ be a prime number. Let $\vec{s} \in {\mathbb Z}^n$ be the secret-key. 
An LWE ciphertext for a message $m \in \{0,1\}$ is a vector $\vec{c} \in {\mathbb F}_q^n$ such that
$$ \langle \vec{c},\vec{s} \rangle= e + m \cdot \lfloor q/2 \rceil \pmod{q}$$
where for the error $e$, we can take the centered binomial distribution $\chi$ with
parameter $\kappa$. 

One can let $e=h(u)-h(v)$
where $u,v \leftarrow \{0,1\}^\kappa$, where $h$ is the Hamming weight function.

In [1]:
def hw(x):
  return sum(x.digits(base=2))

def binom(kappa=2):
  return hw(ZZ.random_element(2^kappa))-hw(ZZ.random_element(2^kappa))

For simplicity, we first consider the symmetric scheme. We consider a binary secret-key $\vec{s}=(s_1,\cdots,s_n)$, with $s_1=1$.

In [2]:
def genKey(n=10,kappa=2,nq=6):
  q=next_prime(2^nq)
  s=vector(ZZ,[1]+[ZZ.random_element(2) for i in range(n-1)])
  return kappa,q,s

In [3]:
kappa,q,s=genKey()
"kappa=%d, q=%d, s=%s" % (kappa,q,s.__str__())

'kappa=2, q=67, s=(1, 1, 0, 0, 0, 0, 0, 1, 0, 1)'

An LWE ciphertext is a vector of elements in ${\mathbb Z}_q$. We have taken $s_1=1$, so 
$$\langle \vec{c},\vec{s} \rangle = c_1 + \sum\limits_{i=2}^n c_i \cdot s_i = e + m \cdot \lfloor q/2 \rceil \pmod{q}$$

In [4]:
def LWEencSym(mes,q,kappa,s):
  a=vector(GF(q),[ZZ.random_element(q) for j in range(s.length())])
  a[0]=-a[1:]*s[1:]+binom(kappa)+ZZ((q-1)/2)*mes
  return a

Decryption is performed by computing
$m={\sf th}(\langle \vec{c},\vec{s} \rangle \bmod q)$, where ${\sf th}(x)=1$ if $q/4 \leq x \leq 3q/4$, and $0$ otherwise.

In [5]:
def th(x,q):
  if x>=q/4 and x<3*q/4: return 1
  return 0

def LWEdecrypt(c,q,s):
  return th((c*s).lift(),q)

To encrypt from the public-key, one can use a matrix 
${\bf A} \in {\mathbb F}_q^{\ell \times n}$ of row vectors  ${\vec a}_i \in {\mathbb F}_q^n$, such that 
$$ \langle \vec{a}_i,\vec{s} \rangle= e_i \pmod q$$
for $e_i \leftarrow \chi$, for all $1 \leq i \leq \ell$.

This can be written ${\bf A} \cdot \vec{s}= \vec{e} \pmod{q}$.
Therefore, every row of $\vec{A}$ is an LWE encryption of $0$.

In [6]:
def vecBinom(n,kappa=2):
  return vector([binom(kappa) for i in range(n)])

def genKeyPK(n=10,ell=40,kappa=2,nq=12):
  q=next_prime(2^nq)
  A=Matrix(GF(q),[[ZZ.random_element(q) for j in range(n)] for i in range(ell)])
  s=vector(ZZ,[1]+[ZZ.random_element(2) for i in range(n-1)])
  A[:,0]=-A[:,1:]*s[1:]+vecBinom(ell,kappa)
  return kappa,q,A,s

In [7]:
kappa,q,A,s=genKeyPK()
"kappa=%d, q=%d, s=%s" % (kappa,q,s.__str__())

'kappa=2, q=4099, s=(1, 1, 0, 0, 0, 1, 0, 1, 0, 1)'

To encrypt a  message $m \in \{0,1\}$, one generates a linear
combination of the row vectors ${\vec a}_i$: 
\begin{align*}
\vec{c}  =\left\lfloor \frac{q}{2} \right\rceil \cdot
(m,0,\ldots,0)+ \vec{u} \cdot \vec{A} \pmod q 
\end{align*}
where $\vec{u} \leftarrow \chi^\ell$.

In [8]:
# mes is the message
def LWEenc(mes, A, kappa, q):
    # Generates vec{u} <- χ^l
    u = vecBinom(A.nrows(), kappa) #where A.nrows()=l
    # Based on the formula above 
    c = vector((q // 2) * mes for _ in range(A.ncols())) + u * A
    c = c % q
    return c

def testLWE():
    message = ZZ.random_element(2)
    print("Original Message:", message)

    # Public-key encryption
    c_pk = LWEenc(message, A, kappa, q)
    # Public-key decryption
    decrypted_pk = LWEdecrypt(c_pk, q, s)
    print("Encrypted Message:", c_pk)
    print("Decrypted Message:", decrypted_pk)
    
testLWE()

Original Message: 0
Encrypted Message: (511, 828, 3125, 1960, 1959, 3777, 2396, 3914, 1390, 3261)
Decrypted Message: 0


## RLWE encryption

For RLWE encryption, we replace ${\mathbb Z}_q$ by 
the polynomial ring $R_q={\mathbb Z}_q[x] / <x^k+1>$, where $k$
is a power of $2$. 

 Addition and multiplication of polynomials are performed modulo
  $x^k+1$ and prime $q$.
 We can take $m \in R_2={\mathbb Z}_2[x] / <\!x^k+1\!>$.

In [9]:
def genRings(k=8,nq=6):
  R=ZZ['x']
  p=R.gen()^k+1
  q=next_prime(2^nq)
  Rq=R.change_ring(GF(q))
  return R,k,p,q,Rq

R,k,p,q,Rq=genRings()
print(f"R:{R},k:{k},p:{p},q:{q},Rq:{Rq}")

R:Univariate Polynomial Ring in x over Integer Ring,k:8,p:x^8 + 1,q:67,Rq:Univariate Polynomial Ring in x over Finite Field of size 67


The public-key is $(a,t)$ with $t=a \cdot s+e$
for random $a$ in $R_q$ and small $s$, $e$ in $R$.

In [10]:
def binomialRLWE(R,k,kappa):
  return R([binom(kappa) for i in range(k)])

def genKeyRLWE(kappa=2,k=8,nq=12):
  R,k,p,q,Rq=genRings(k,nq)
  s=binomialRLWE(R,k,kappa)
  a=Rq([ZZ.random_element(q) for i in range(k)])
  e=binomialRLWE(R,k,kappa)
  t=(a*s+e) % p
  return q,Rq,R,p,kappa,k,a,t,s

In [11]:
q,Rq,R,p,kappa,k,a,t,s=genKeyRLWE()
print(f"q:{q},Rq:{Rq},R:{R},p:{p},kappa:{kappa},k:{k},a:{a},t:{t},s:{s}")

q:4099,Rq:Univariate Polynomial Ring in x over Finite Field of size 4099,R:Univariate Polynomial Ring in x over Integer Ring,p:x^8 + 1,kappa:2,k:8,a:2621*x^7 + 1986*x^6 + 1432*x^5 + 21*x^4 + 2786*x^3 + 24*x^2 + 3455*x + 772,t:2197*x^7 + 1610*x^6 + 2840*x^5 + 273*x^4 + 1602*x^3 + 1491*x^2 + 2077*x + 564,s:x^5 + x^4 + x^3 + x - 1


To encrypt
a message $m \in R_2$, compute
$ c=(a \cdot r+e_1,~t \cdot r+e_2 + \lfloor q/2 \rceil m)$,
for small $e_1$, $e_2$ and $r$ in $R$. 

To decrypt,
compute $m={\sf th}(v- s \cdot u)$.
\begin{align*}
 v- { s} \cdot u & =
     { t} \cdot r+e_2 + \lfloor q/2 \rceil m
     - { s} \cdot ( { a} \cdot
     r+e_1) \\
     & = ({ t} -{ a}
     \cdot { s} ) \cdot r+e_2 + \lfloor q/2 \rceil m
     - { s} \cdot e_1 \\
     & =\lfloor q/2 \rceil m +
\underbrace{
     { e} \cdot  r + e_2
     - { s} \cdot e_1}_{\text{small}}
\end{align*}

In [12]:
def encryptRLWE(m,R,q,p,kappa,k,a,t):
    
    # Adds random noise
    e1 = binomialRLWE(R, k, kappa) 
    e2 = binomialRLWE(R, k, kappa) 
    # Small random polynomial
    r = binomialRLWE(R, k, kappa) 

    # mod p (i.e., %p) ensures the result is within the polynomial ring
    c0 = (a*r+e1)%p
    c1 = (t*r+e2+((q // 2) * m))%p #(q // 2) adds noise
    
    return c0,c1


def decryptRLWE(c,R,q,p,s):
    c0, c1 = c 
    u = (s*c0)%p
    v = ((c1-u)%p)//(q // 2) #//(q // 2) removes the noise
    
    # Apply th() to each term in v
    m = v.map_coefficients(lambda x: th(x.lift(), q))
    m = Rq(m) #makes sure we're working with a polynomial ring
    
    return m


def testRLWE():
    message = Rq([ZZ.random_element(q) for i in range(k)])
    print(f"Original Message: {message}")

    encrypted_message = encryptRLWE(message,R,q,p,kappa,k,a,t)
    print(f"Encrypted Message: {encrypted_message}")
    
    decrypted_message = decryptRLWE(encrypted_message, R, q, p, s)
    print(f"Decrypted Message: {decrypted_message}")


testRLWE()

Original Message: 1507*x^7 + 3203*x^6 + 2245*x^5 + 1331*x^4 + 1837*x^3 + 3119*x^2 + 2822*x + 1262
Encrypted Message: (1320*x^7 + 3567*x^6 + 2389*x^5 + 2323*x^4 + 2849*x^3 + 2798*x^2 + 2898*x + 1552, 3312*x^7 + 3714*x^6 + 4010*x^5 + 942*x^4 + 1851*x^3 + 1510*x^2 + 1257*x + 1230)
Decrypted Message: x^7 + x^5 + x^4 + x^3 + x + 1


The error $e$ must be kept secret, otherwise we can recover the secret-key $s$ using
$$ s=(t-e)/a \pmod{x^k+1,q}$$

In [13]:
def from_e():
    s=binomialRLWE(R,k,kappa)
    print("s = %s" % str(s))
    a=Rq([ZZ.random_element(q) for i in range(k)])
    e=binomialRLWE(R,k,kappa)
    t=(a*s+e) % p
    
    # show how to recover s from the knowledge of a, t and e.
    s_recover = ((t - e) * a.inverse_mod(p)) % p
    s_recover = Rq(s_recover)
    
    print("Recovered s = %s" % str(s_recover))


from_e()

s = x^5 - x^4 + x^3 + x
Recovered s = x^5 + 4098*x^4 + x^3 + x
